# 1. Import Libraries and Load Dataset

In [ ]:
%%capture
#!git clone https://github.com/huggingface/transformers.git
!pip install datasets evaluate transformers[sentencepiece]
!pip install pyvi
!pip install gdown

In [ ]:
#%%capture

import gdown

!gdown 1jWDnP1xF01_pmeJYsRHxSVBkLZ0I9LAt #utils T5 for VQA

In [ ]:
from PIL import Image

import requests

import os
import shutil
import torch

from urllib.request import urlopen
from PIL import Image
import timm
import tqdm
import torch
from IPython.display import FileLink

In [ ]:
import json

train_json = json.load(open('/kaggle/input/dsc24-vimmsd/vimmsd-train.json', encoding='utf-8'))
dev_json = json.load(open('/kaggle/input/dsc24-vimmsd/vimmsd-public-test.json', encoding='utf-8'))

# Image feature extraction

In [ ]:
# images = []

# for key, item in train_json.items():
#     images.append(item['image'])

# for key, item in dev_json.items():
#     images.append(item['image'])

In [ ]:
# source_all = ['/kaggle/input/dsc24-vimmsd/dev-images/dev-images',
#               '/kaggle/input/dsc24-vimmsd/train-images/train-images']

# destination = './images'
# #os.mkdir(destination)

# for source in source_all:
#     allfiles = os.listdir(source)
#     for f in allfiles:
#         src_path = os.path.join(source, f)
#         dst_path = os.path.join(destination, f)
#         shutil.copy(src_path, dst_path)

## BEiT

In [ ]:
# model = timm.create_model(
#     'beitv2_base_patch16_224.in1k_ft_in22k_in1k',
#     pretrained=True,
#     num_classes=0,  # remove classifier nn.Linear
# ).to('cuda')

# model = model.eval()
# data_config = timm.data.resolve_model_data_config(model)
# transforms = timm.data.create_transform(**data_config, is_training=False)

# img_w = {}

# def batch(iterable, n=1):
#     l = len(iterable)
#     for ndx in range(0, l, n):
#         yield iterable[ndx:min(ndx + n, l)]

In [ ]:
# p = '/kaggle/working/images/'

# img_w = {}

# # get features for images, we will do this 3 images at a time to reduce time

# for x in tqdm.notebook.tqdm(batch(images, 3),total=int(len(images)/3)):
#     img = [Image.open(p + v).convert('RGB') for v in x]
#     print(x)

#     with torch.no_grad():
#         img  = torch.stack([transforms(i) for i in img]).to('cuda')
#         output = model.forward_features(img)[:,1:,:]

#     tmp_img_w = {k:v for k,v in zip(x,output)}

#     img_w.update(tmp_img_w)

#     del output
#     del tmp_img_w
#     del img
#     torch.cuda.empty_cache()

# torch.save(img_w, '/kaggle/working/beitv2-b-p.pt') # export for later used
# FileLink('/kaggle/working/beitv2-b-p.pt')

## Load extracted image features

In [ ]:
import torch
from torch.nn.functional import normalize

img_w = torch.load('/kaggle/input/dsc24-vit/vit-b.pt')
# img_w = torch.load('/kaggle/input/lovecat-beitv2-b-p/beitv2-b-p.pt') # already-saved features
len(img_w)

In [ ]:
img_w['ac7931bb887ad853b41675f07595bf04469970d1b099ffc8806a4ceaac7d7940.jpg'].shape

# Model

In [1]:
!git clone https://github.com/HMD191/UIT-DSC-2024.git

Cloning into 'UIT-DSC-2024'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 48 (delta 14), reused 21 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 868.77 KiB | 15.79 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [2]:
%cd UIT-DSC-2024/VisualBERT

/kaggle/working/UIT-DSC-2024/VisualBERT


In [16]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 5.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44090 sha256=a54af7cf48c57c5f2aff769d39058df55729e2e2fcac49bd16fffd3e06d38e9f
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask 2023.9.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-c

FileNotFoundError: [Errno 2] No such file or directory

In [19]:
%%writefile visualbert.py
import numpy as np
import pandas as pd
import pickle

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.functional import softmax

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import VisualBertModel, VisualBertConfig

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix, f1_score

from visual_embeds import *
from utils import *

RANDOM_SEED = 42
MAX_LEN = 64
N_CLASSES = 2
N_EPOCHS = 10
BATCH_SIZE = 32

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
pl.seed_everything(RANDOM_SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class MemesDataset(Dataset):
    '''Wrap the tokenization process in a PyTorch Dataset, along with converting the labels to tensors'''
    
    def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_len: int, visual_embeds):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = max_len
        self.visual_embeds = visual_embeds
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        data_row = self.data.iloc[index]
        text = data_row.caption
        labels = data_row.misogynous

        tokens = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = torch.tensor(tokens["input_ids"]).flatten()
        attention_mask = torch.tensor(tokens["attention_mask"]).flatten()

        visual_embedding = self.visual_embeds[index].to('cpu')
        visual_attention_mask = torch.ones(visual_embedding.shape[:-1], dtype=torch.float)
        visual_token_type_ids = torch.ones(self.visual_embeds.shape[:-1], dtype=torch.long)

        return dict(
            input_ids=input_ids,
            attention_mask=attention_mask,
            visual_embedding=visual_embedding,
            visual_attention_mask=visual_attention_mask,
            visual_token_type_ids=visual_token_type_ids,
            labels=torch.tensor(labels).float()
        )


class MemesDataModule(pl.LightningDataModule):
    '''
    1- Split the dataset into training and validation dataset
    2- Create Dataloaders from Datasets (Divide data into batches)
    '''

    def __init__(self, df, tokenizer, visual_embeds, batch_size=32, max_len=64):
        super().__init__()
        self.batch_size = batch_size
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.visual_embeds = visual_embeds
  
    def setup(self, stage=None):
        self.dataset = MemesDataset(self.df, self.tokenizer, self.max_len, self.visual_embeds)
        self.train_dataset, self.val_dataset = train_test_split(self.dataset, test_size=0.1) # Split the dataset into training and validation datasets
  
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size = self.batch_size,
            shuffle=True,
            num_workers=3
        )
  
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size = self.batch_size,
            num_workers=3
        )
  
    def test_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size = self.batch_size,
            num_workers=3
        )


class MemesClassifier(pl.LightningModule):
  '''Wrap the training of VisualBERT model to classify memes'''

  def __init__(self, n_classes, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-vqa-coco-pre', visual_embedding_dim=1024)
    self.model = VisualBertModel(self.configuration)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.CrossEntropyLoss()
    self.dropout = nn.Dropout(0.2)
    self.classifier = nn.Linear(self.model.config.hidden_size, n_classes)
  
  
  def forward(self, input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels=None):
    output = self.model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        visual_embeds=visual_embeds,
                        visual_attention_mask=visual_attention_mask,
                        visual_token_type_ids=visual_token_type_ids
    )
    
    output = self.dropout(output.pooler_output)
    output = self.classifier(output)

    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)

    return loss, output
  
  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    visual_embeds = batch['visual_embedding'].to(device)
    visual_attention_mask = batch['visual_attention_mask'].to(device)
    visual_token_type_ids = batch['visual_token_type_ids'].to(device)

    labels = batch['labels'].type(torch.LongTensor).to(device)
    
    loss, outputs = self(input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels)
    self.log('train_loss', loss, prog_bar=True, logger=True)

    return {"loss":loss, 'predictions':outputs, 'labels':labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    visual_embeds = batch['visual_embedding']
    visual_attention_mask = batch['visual_attention_mask']
    visual_token_type_ids = batch['visual_token_type_ids'].to(device)
    labels = batch['labels'].type(torch.LongTensor).to(device)
    
    loss, outputs = self(input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels)
    self.log('val_loss', loss, prog_bar=True, logger=True)

    return loss
  
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=5e-5)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=self.n_warmup_steps,
        num_training_steps=self.n_training_steps
    )

    return dict(
        optimizer=optimizer,
        lr_scheduler=dict(
            scheduler=scheduler,
            interval='step'
        )
    )


def train_model(model, df, tokenizer, visual_embeds):
    checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints",
        filename="best-checkpoint",
        save_top_k=1,
        verbose=True,
        monitor="val_loss",
        mode="min",
    )
    
    logger = TensorBoardLogger("lightning_logs", name="memes-text")
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

    data_module = MemesDataModule(
        df=df,
        tokenizer=tokenizer,
        visual_embeds=visual_embeds,
        batch_size=BATCH_SIZE,
        max_len=MAX_LEN
    )

    trainer = pl.Trainer(
        logger=logger,
        callbacks=[early_stopping_callback, checkpoint_callback],
        max_epochs=N_EPOCHS,
        gpus=1,
        progress_bar_refresh_rate=10
    )

    trainer.fit(model, data_module)


def evaluate_model(test_dataset, checkpoint):
    trained_model = MemesClassifier.load_from_checkpoint(
        checkpoint,
        n_classes=2
    ).to(device)

    trained_model.eval()
    trained_model.freeze()

    predictions = []
    labels = []
    for item in tqdm(test_dataset):
        _, prediction = trained_model(
            item["input_ids"].unsqueeze(dim=0).to(device),
            item["attention_mask"].unsqueeze(dim=0).to(device),
            item["visual_embedding"].unsqueeze(dim=0).to(device),
            item['visual_attention_mask'].unsqueeze(dim=0).to(device),
            item['visual_token_type_ids'].unsqueeze(dim=0).to(device)
        )
        predictions.append(prediction.flatten())
        labels.append(item["labels"].int())
    
    predictions = torch.stack(predictions).detach().cpu()
    labels = torch.stack(labels).detach().cpu()

    _, preds = torch.max(torch.tensor(predictions), dim=1)
    
    f1_macro = f1_score(labels, preds , average="macro")
    f1_micro = f1_score(labels, preds , average="micro")
    accuracy = accuracy(predictions, labels)

    return f1_macro, f1_micro, accuracy
    

def main(): 
    df = pd.read_csv('./Data/your_training.csv')
    df.text = np.array([preprocess_text(text) for text in df.text])

    visual_embeds = generate_visual_embeds(
        images_file_names=df.file_name,
        images_path='./Data/Images', # You can put your images in this folder or change the path
        cfg_path='COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
    )
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    steps_per_epoch = 8000 // BATCH_SIZE
    total_training_steps = steps_per_epoch * N_EPOCHS
    warmup_steps = total_training_steps // 10

    model = MemesClassifier(
        n_classes=N_CLASSES,
        n_training_steps=total_training_steps,
        n_warmup_steps=warmup_steps,
    ).to(device)

    train_model(
        model=model, 
        df=df,
        tokenizer=tokenizer,
        visual_embeds=visual_embeds
    )

    test_df = pd.read_csv('./Data/your_test.csv')
    test_dataset = MemesDataset(
        test_df, 
        tokenizer, 
        MAX_LEN
    )

    f1_macro, f1_micro, accuracy = evaluate_model(
        test_dataset=test_dataset,
        checkpoint='./checkpoints/best-checkpoint.ckpt'
    )


if __name__ == '__main__':
    main()


Overwriting visualbert.py


In [16]:
import pickle
import re
import cv2

In [17]:
%%writefile requirements.txt
torch
torchMetrics
transformers
pandas
pytorch-lightning
matplotlib
seaborn
numpy
nltk


Overwriting requirements.txt


In [18]:
!pip install -r requirements.txt

In [14]:
from visualbert import MemesDataset, MemesClassifer

ModuleNotFoundError: No module named 'detectron2'

In [ ]:
# df = pd.read_csv('./Data/your_training.csv') --> train_df

visual_embeds = generate_visual_embeds(
    images_file_names=df.image,
    images_path='/kaggle/input/dsc24-vimmsd/train-images/train-images', # You can put your images in this folder or change the path
    cfg_path='COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
)

tokenizer = BertTokenizer.from_pretrained('uitnlp/visobert')

steps_per_epoch = 8000 // 8
total_training_steps = steps_per_epoch * 2
warmup_steps = total_training_steps // 10

model = MemesClassifier(
    n_classes=4,
    n_training_steps=total_training_steps,
    n_warmup_steps=warmup_steps,
).to(device)

train_model(
    model=model, 
    df=df,
    tokenizer=tokenizer,
    visual_embeds=visual_embeds
)

val_dataset = MemesDataset(
    val_df, 
    tokenizer, 
    MAX_LEN
)

f1_macro, f1_micro, accuracy = evaluate_model(
    test_dataset=val_dataset,
    checkpoint='./checkpoints/best-checkpoint.ckpt'
)

In [ ]:
# load checkpoint and predict

## Prepare datasets

In [ ]:
emoji_file_path = '/kaggle/input/datasets-preprocesing/emoji_to_vietnamese.json'
stopword_path = '/kaggle/input/datasets-preprocesing/vietnamese-stopwords.txt'

def load_resources(stopword_path, emoji_file_path):
    # Đọc stopword từ file txt
    with open(stopword_path, 'r', encoding='utf-8') as f:
        stopwords = set(f.read().splitlines())

    # Đọc emoji từ file JSON
    with open(emoji_file_path, 'r', encoding='utf-8') as emoji_file:
        emoji_dict = json.load(emoji_file)

    return stopwords, emoji_dict

stopwords, emoji_dict = load_resources(stopword_path, emoji_file_path)

In [ ]:
import pandas as pd

train_df = pd.DataFrame(train_json).T
test_df = pd.DataFrame(dev_json).T # public test

In [ ]:
train_df['image_id'] = train_df['image'].astype(str)
test_df['image_id'] = test_df['image'].astype(str)

train_df['caption'] = train_df['caption'].astype(str)
test_df['caption'] = test_df['caption'].astype(str)

train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)

train_df.drop(columns=['image'], inplace=True)
test_df.drop(columns=['image'], inplace=True)

train_df.head()

### Features enrichment

In [ ]:
!gdown 1q7_-PEQQ6IR3Ortz45vEiSiOnweRJ40H # cap train
!gdown 1wldmw8IJgX-nK2_yfo8fLx55KfWGIZhp # cap dev 

In [ ]:
!gdown 1AnM0RUMfyGYWaiUgafufEKMjB8zo5dUt # object reg train
!gdown 1sk2vJutRJLCUwQ6ZKJQeYFKpvdkjUiBs # object reg dev

In [ ]:
!gdown 1nh3y-lXq2CEc_rwzeTqGIU69VZA4eVEn # OCR dev
!gdown 1YSn-dWwprc0nhOgRUIj5aFPaW9lxKWZT

In [ ]:

# Open JSON files with utf-8 encoding to handle non-ASCII characters
with open('/kaggle/working/vi_train_captions.json', encoding='utf-8') as f:
    cap_train = json.load(f)

with open('/kaggle/working/vi_dev_captions.json', encoding='utf-8') as f:
    cap_test = json.load(f)

with open('/kaggle/working/objects-recognition-train.json', encoding='utf-8') as f:
    obj_train = json.load(f)

with open('/kaggle/working/objects-recognition-dev.json', encoding='utf-8') as f:
    obj_test = json.load(f)

with open('/kaggle/working/ocr-results-dev.json', encoding='utf-8') as f:
    ocr_test = json.load(f)

with open('/kaggle/working/ocr-results-train.json', encoding='utf-8') as f:
    ocr_train = json.load(f)

In [ ]:
def json_to_df(json):
    df = pd.DataFrame(json)
    df['image_id'] = df['image'].astype(str)
    df.drop(columns=['image'], inplace=True)

    return df

In [ ]:
for item in ocr_train:
    item["OCR"] = ", ".join(item["OCR"])
for item in ocr_test:
    item["OCR"] = ", ".join(item["OCR"])

In [ ]:
cap_test_df = json_to_df(cap_test)

cap_train_df = json_to_df(cap_train)

obj_train_df = json_to_df(obj_train)

obj_test_df = json_to_df(obj_test)

ocr_train_df = json_to_df(ocr_train)

ocr_test_df = json_to_df(ocr_test)

In [ ]:
obj_train_df['object_recognition'] = obj_train_df['object_recognition'].apply(lambda x: "Trong hình có " + x + ". " if len(x) > 0 else "")
obj_test_df['object_recognition'] = obj_test_df['object_recognition'].apply(lambda x: "Trong hình có " + x + ". " if len(x) > 0 else "")
obj_test_df.head()

In [ ]:
ocr_train_df['OCR'] = ocr_train_df['OCR'].apply(lambda row: "Chữ trong hình là " + row + ". " if len(row) > 0 else "")
ocr_test_df['OCR'] = ocr_test_df['OCR'].apply(lambda row: "Chữ trong hình là " + row + ". " if len(row) > 0 else "")

In [ ]:
def enrich(df1, df2, add_field):
    temp = df2.set_index('image_id')
    
    df1['caption'] = df1.apply(
        lambda row: row['caption'] + ' ' + temp.loc[row['image_id'], add_field]
        if row['image_id'] in temp.index else row['caption'], axis=1
    )

    return df1


In [ ]:
train_df['caption'] = train_df['caption'].apply(lambda x: x[:150] if len(x) > 150 else x)
test_df['caption'] = test_df['caption'].apply(lambda x: x[:150] if len(x) > 150 else x)

train_df = enrich(train_df, ocr_train_df, 'OCR')
test_df = enrich(test_df, ocr_test_df, 'OCR')


train_df = enrich(train_df, cap_train_df, 'caption')
test_df = enrich(test_df, cap_test_df, 'caption')

train_df = enrich(train_df, obj_train_df, 'object_recognition')
test_df = enrich(test_df, obj_test_df, 'object_recognition')

In [ ]:
train_df.head()

### Text preprocessing

In [ ]:
import re
def preprocess_text(text):
    def remove_stopwords(text):
        return text
    def replace_emojis(text):
        for emoji, description in emoji_dict.get('emoji', {}).items():
            text = text.replace(emoji, description)  # Thay thế emoji bằng mô tả
        return text

    def replace_emoticons(text):
        for emoticon, meaning in emoji_dict.get('biểu_tượng', {}).items():
            emoticon_pattern = re.escape(emoticon) + r"{1,}"
            text = re.sub(emoticon_pattern, meaning, text)
        return text

    def normalize_text(text):
        text = text.lower()  # Chuyển thành chữ thường
        text = re.sub(r'(?<=\w)[\/\.\-\_,\\](?=\w)', '', text)  # Loại bỏ dấu chấm hoặc gạch nối trong từ
        return text

    text = replace_emojis(text)       # Thay thế emoji
    text = replace_emoticons(text)    # Thay thế biểu cảm
    text = normalize_text(text)       # Chuẩn hóa văn bản
    text = remove_stopwords(text)
    return text

In [ ]:
train_df['caption'] = train_df['caption'].astype(str)
test_df['caption'] = test_df['caption'].astype(str)

In [ ]:
train_df['caption'] = train_df['caption'].apply(preprocess_text)
test_df['caption'] = test_df['caption'].apply(preprocess_text)

In [ ]:
train_df['caption'].iloc[2003]


In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['label'])  # Features
y = train_df['label']  # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_test, y_test], axis=1)

In [ ]:
print(train_df.shape, val_df.shape, test_df.shape)

### Convert to Dataset

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=400, truncation=True, padding=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=32, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    model_inputs["image_id"] = examples["image_id"]
    
    return model_inputs

In [ ]:
dict_obj = {}

dict_obj['inputs'] = train_df['caption'].astype(str).tolist()  # Convert Series to list
dict_obj['labels'] = train_df['label'].astype(str).tolist()    # Convert Series to list
dict_obj['image_id'] = train_df['image_id'].astype(str).tolist()
train_dataset = Dataset.from_dict(dict_obj)
tokenized_train_datasets = train_dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

dict_obj = {}
dict_obj['inputs'] = val_df['caption'].astype(str).tolist()  # Convert Series to list
dict_obj['labels'] = val_df['label'].astype(str).tolist()    # Convert Series to list
dict_obj['image_id'] = val_df['image_id'].astype(str).tolist()
val_dataset = Dataset.from_dict(dict_obj)
tokenized_val_datasets = val_dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)


# Train

In [ ]:
import random

import numpy as np

import torch

def set_SEED():
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
import os
from transformers.optimization import Adafactor, AdafactorSchedule
from transformers import DataCollatorForSeq2Seq
from utils import DataCollatorForSeq2Seq

os.environ["WANDB_DISABLED"] = "True"
set_SEED()

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

#Adam
training_args = Seq2SeqTrainingArguments(output_dir="./checkpoint",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=1,
                                      learning_rate=2.5e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=8,
                                      per_device_eval_batch_size=8,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="steps",
                                      evaluation_strategy="steps",
                                      save_total_limit=5,
                                      eval_steps=100,
                                      logging_steps = 100,
                                      save_steps=100,
                                      load_best_model_at_end= True,
                                      fp16=True,
                                      seed=42,
                                      )

In [ ]:
tokenized_train_datasets

In [ ]:
tokenized_val_datasets

In [ ]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
import matplotlib.pyplot as plt
import json

# save loss
log_history = {'log_history':trainer.state.log_history}

with open('logs.json', 'w', encoding='utf-8') as f:
    json.dump(log_history, f, ensure_ascii=False, indent=4)

In [ ]:
from matplotlib import pyplot

train_loss = {i['step']:i['loss'] for i in log_history['log_history'] if 'loss' in i.keys()}
eval_loss = {i['step']:i['eval_loss'] for i in log_history['log_history'] if 'eval_loss' in i.keys()}
plt.plot(list(train_loss.values()))
plt.plot(list(eval_loss.values()))

plt.yscale('log')
plt.legend(['train','val'])

# Load checkpoints and predict



In [ ]:
torch.cuda.empty_cache()
# model = T5ForConditionalGeneration.from_pretrained("/kaggle/working/checkpoint/checkpoint-1200")
# model.to('cuda')
# model.add_imgw(img_w)

In [ ]:
from datasets import Dataset

dict_obj = {}
dict_obj['inputs'] = test_df['caption']
dict_obj['labels'] =  test_df['label']
dict_obj['image_id'] = test_df['image_id']
test_dataset = Dataset.from_dict(dict_obj)
tokenized_test_datasets = test_dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
tokenized_test_datasets

In [ ]:
import torch 
import numpy as np
from datasets import load_metric
metrics = load_metric('accuracy')

torch.cuda.empty_cache()
max_target_length = 10
dataloader = torch.utils.data.DataLoader(tokenized_test_datasets, collate_fn=data_collator, batch_size=4) #replace tokenized_dev_datasets with tokenized_test_datasets

predictions = []
references = []

for i, batch in enumerate(tqdm(dataloader)):
    # greedy search
    outputs = model.generate(image_id = batch['image_id'],
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
        return_dict_in_generate=True, output_attentions=True,)

    #beam search for now   
    # outputs = model.generate(
    #     image_id = np.repeat(batch['image_id'], 7),
    #     input_ids=batch['input_ids'].to('cuda'),
    #     max_length=max_target_length,
    #     attention_mask=batch['attention_mask'].to('cuda'),
    #     return_dict_in_generate=True, output_attentions=True,
    #     num_beams=7,
    #     no_repeat_ngram_size=2)
    
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs.sequences]
        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        # actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]

    predictions.extend(outputs)


In [ ]:
len(predictions)

In [ ]:
def postprocessing(t):
    t = t.lower()
    t = t.replace('\\','')
    return t

test_predicted = {k:postprocessing(i) for k,i in zip(dev_json.keys(),predictions)}

In [ ]:
result_json = {
    "results": test_predicted,
    "phase": "dev"
}

In [ ]:

with open('results.json', 'w') as fp:
    json.dump(result_json, fp,ensure_ascii=True,indent=True)

# Finetune for classification task